In [1]:
import cv2
import pandas as pd
import numpy as np

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import os
import random
import gc
import tensorflow as tf

train_dir = 'train (2).csv'
test_dir = 'test (2).csv'

train = pd.read_csv(train_dir,sep='\t')
test = pd.read_csv(test_dir,sep='\t')
#print(train)
temp_train_imgs = train['imageid']
temp_test_imgs = test['imageid']

mystring = '.jpg'
train_imgs = [str(s) + mystring for s in temp_train_imgs]
test_imgs = [str(s) + mystring for s in temp_test_imgs]


"""#print(the_train_imgs)

import matplotlib.image as mping

for ima in train_imgs[0:3]:
    img=mping.imread('images/' + ima)
    imgplot = plt.imshow(img)
    plt.show()
"""

X = [] #images

print('hi')
for image in train_imgs:
    #read in each image as greyscale
    X.append(cv2.resize(cv2.imread('images/' +image), (60, 80), interpolation = cv2.INTER_CUBIC))
    
print('done')

hi
done


In [2]:
Y = pd.get_dummies(train.label)
print(Y)
#print(X)

       Bags  Bottomwear  Eyewear  Fragrance  Innerwear  Jewellery  Makeup  \
0         1           0        0          0          0          0       0   
1         0           0        0          0          0          0       0   
2         0           0        0          0          0          0       0   
3         0           0        0          0          0          0       0   
4         0           1        0          0          0          0       0   
...     ...         ...      ...        ...        ...        ...     ...   
40436     0           0        0          0          0          0       0   
40437     0           0        0          0          0          0       0   
40438     0           1        0          0          0          0       0   
40439     0           0        0          0          0          0       0   
40440     0           0        0          0          0          0       0   

       Others  Sandal  Shoes  Topwear  Wallets  Watches  
0           0    

In [3]:
X = np.array(X)
Y = np.array(Y)
X = np.true_divide(X, 255)
print(X.shape)
print(Y.shape)

(40441, 80, 60, 3)
(40441, 13)


In [4]:
#replicate above for testing data
X_test = []
for image in test_imgs:
    #read in each image as color
    X_test.append(cv2.resize(cv2.imread('images/' +image), (60, 80), interpolation = cv2.INTER_CUBIC))
    
Y_test = pd.get_dummies(test.label)
print('done')

done


In [5]:
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_test = np.true_divide(X_test, 255)
print(X_test.shape)
print(Y_test.shape)

(4000, 80, 60, 3)
(4000, 13)


In [14]:
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Model

In [15]:
# load model and specify a new input shape for images
input_t = keras.Input(shape=(80,60,3))
pre_trained_model = keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_tensor = input_t)
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 80, 60, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 86, 66, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 40, 30, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 40, 30, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [16]:
model = models.Sequential()
model.add(pre_trained_model)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(13, activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(X, Y, epochs=10, batch_size = 64, verbose = 1)

Epoch 1/10
632/632 [==============================] - 182s 284ms/step - loss: 0.8169 - accuracy: 0.7620
Epoch 2/10
632/632 [==============================] - 182s 288ms/step - loss: 0.4615 - accuracy: 0.8579
Epoch 3/10
295/632 [=============>................] - ETA: 1:36 - loss: 0.4279 - accuracy: 0.8662

KeyboardInterrupt: 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, epochs=40, batch_size = 64, verbose = 1)

Epoch 1/50
632/632 [==============================] - 174s 270ms/step - loss: 0.8194 - accuracy: 0.7620
Epoch 2/50
632/632 [==============================] - 171s 271ms/step - loss: 0.4614 - accuracy: 0.8539
Epoch 3/50
632/632 [==============================] - 170s 269ms/step - loss: 0.4012 - accuracy: 0.8727
Epoch 4/50
632/632 [==============================] - 176s 279ms/step - loss: 0.3794 - accuracy: 0.8752
Epoch 5/50
632/632 [==============================] - 171s 271ms/step - loss: 0.3461 - accuracy: 0.8875
Epoch 6/50
632/632 [==============================] - 173s 274ms/step - loss: 0.3264 - accuracy: 0.8892
Epoch 7/50
632/632 [==============================] - 172s 272ms/step - loss: 0.3199 - accuracy: 0.8943
Epoch 8/50
632/632 [==============================] - 170s 270ms/step - loss: 0.3069 - accuracy: 0.8970
Epoch 9/50
632/632 [==============================] - 170s 269ms/step - loss: 0.2933 - accuracy: 0.9018
Epoch 10/50
632/632 [==============================] - 173s 273m

In [ ]:
model.save("updated_model")
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
#load the model back in if needed
reconstructed_model = keras.models.load_model("updated_model")
model.summary()